<a href="https://colab.research.google.com/github/atoney-CSET/github-playground/blob/main/overlap_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#authenticate for BQ access
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
#Import Packages
import pandas as pd
from google.cloud import bigquery


In [3]:
method_labels = {
    "approach_1_keyword_search": "Keyword Search",
    "approach_2_classifier_identification": "arXiv Classifier",
    "approach_3_ai_cluster_assignment": "Map of Science",
    "approach_4_mag": "Fields of Study",
}

agreement_queries = {
    method_label: f"""
with predictions as (
  select 
    merged_id, 
    max(approach_1_keyword_search) as approach_1_keyword_search,
    max(approach_2_classifier_identification) as approach_2_classifier_identification,
    max(approach_3_ai_cluster_assignment) as approach_3_ai_cluster_assignment,
    max(approach_4_mag) as approach_4_mag,
  from `cs2078_sandbox.researching_ai_analysis_subset`
  where {method_col} = 1
  group by merged_id
)
select 
  sum(approach_1_keyword_search) / count(*) as `Keyword Search`,
  sum(approach_2_classifier_identification) / count(*) as `arXiv Classifier`,
  sum(approach_3_ai_cluster_assignment) / count(*) as `Map of Science`,
  sum(approach_4_mag) / count(*) as `Fields of Study`,
  sum(if(
    approach_1_keyword_search + 
    approach_2_classifier_identification +
    approach_3_ai_cluster_assignment +
    approach_4_mag = 1, 1, 0)) / count(*) as `Only`,
from predictions
""" for method_col, method_label in method_labels.items()}
agreement = pd.concat({
    method: pd.read_gbq(query, project_id="gcp-cset-projects")
    for method, query in agreement_queries.items()
})

print(agreement.transpose())

                 Keyword Search arXiv Classifier Map of Science  \
                              0                0              0   
Keyword Search         1.000000         0.501651       0.499360   
arXiv Classifier       0.546267         1.000000       0.703524   
Map of Science         0.347654         0.449789       1.000000   
Fields of Study        0.494927         0.563334       0.641886   
Only                   0.282594         0.227069       0.099088   

                 Fields of Study  
                               0  
Keyword Search          0.453142  
arXiv Classifier        0.561647  
Map of Science          0.409153  
Fields of Study         1.000000  
Only                    0.220916  
